<a href="https://colab.research.google.com/github/x110/MyFastaiNotebooks/blob/master/lang_model_arxiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language modeling

## Set up google colab

In [0]:
!pip install git+https://github.com/fastai/fastai.git

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


TIMEOUT: ignored

In [0]:
#drive.mount("/content/drive", force_remount=True)

## Get your data ready for modeling

In [0]:
from fastai.text import *

In [0]:
my_tok2=SpacyTokenizer("en")
w = ['<SUMM>','<CAT>','<TITLE>','<BR />','<BR>']
my_tok2.add_special_cases(w)

In [0]:
PATH='drive/My Drive/myfastai/data/arxiv1/'

In [0]:
FILES = dict(train='train', valid='valid')

In [0]:
#data = TextLMDataBunch.from_folder(f'{PATH}all/',**FILES, min_freq=10)

Since this step can be a bit time-consuming, it's best to save the result with:

In [0]:
#data.save()

This will create a 'tmp' directory where all the computed stuff will be stored. You can then reload those results with:

In [0]:
data = TextLMDataBunch.load(f'{PATH}all/')

## Fine-tuning a language model

We can use the data object we created earlier to fine-tune a pretrained language model. fast.ai has an English model available that we can download. We can create a learner object that will directly create a model, download the pretrained weights and be ready for fine-tuning.



In [0]:
learn = language_model_learner(data, pretrained_model=URLs.WT103_1, drop_mult=0.5)

In [0]:
learn.fit_one_cycle(1, 1e-2)

In [0]:
??learn.fit_one_cycle

In [0]:
learn = language_model_learner(data,
                               emb_sz = 200,   
                               nh = 500,
                               drop_mult=0.5)

In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(1, 3e-3, wd=1e-6)

epoch,train_loss,valid_loss,accuracy
1,0.004069,7.125626,0.143073


In [0]:
!ls

sample_data


In [0]:
learn.fit_one_cycle(1, 3e-3, wd=1e-6)

epoch,train_loss,valid_loss,accuracy
1,0.004610,7.065801,0.139501


In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

TextLMDataBunch;

Train: LabelList
y: LMLabel (0 items)
[]...
Path: /content/drive/My Drive/myfastai/data/arxiv1/all
x: LMTextList (0 items)
[]...
Path: /content/drive/My Drive/myfastai/data/arxiv1/all;

Valid: LabelList
y: LMLabel (0 items)
[]...
Path: /content/drive/My Drive/myfastai/data/arxiv1/all
x: LMTextList (0 items)
[]...
Path: /content/drive/My Drive/myfastai/data/arxiv1/all;

Test: None

In [0]:
learn.predict("This is a review about", n_words=10)

In [0]:
learn.save_encoder('ft_enc')